### TODOs

- [ ] Increase charts size
- [ ] Make notes for the presentation
- [ ] Data analysis
- [ ] Package simulation as a class
- [ ] Plotly instead of pandas plots

# Project: Monte Carlo Markov Chain Simulation

## Business goals:  

1. understand customer behavior  
2. explain customer behavior to non-data staff  
3. optimize staffing so that the queues do not get unnecessary long  

## Supermarket Area

We are using the following model supermarket with six areas: entrance, fruit, spices, dairy, drinks and checkout.

The customers can move between these areas freely. Sooner or later, they will enter the checkout area. Once they do, they are considered to have left the shop.

![Drag Racing](./supermarket.png)

### Load data

In [ ]:
from os import listdir
from os.path import isfile, join

def load_data(day):
    path = './data/'
    r = pd.read_csv(os.path.join(path, day + '.csv'), sep=';', parse_dates=['timestamp'], index_col=['timestamp'])
    return r

df = load_data('monday')
for file in ['tuesday', 'wednesday', 'thursday', 'friday']:
    df_next = load_data(file)
    df_next['customer_no'] = df_next['customer_no'] + df['customer_no'].max()
    df = df.append(df_next)

df

### Enrich data

In [ ]:
def resample_transitions(df):
    df_full = df.groupby(by=['customer_no']).resample('1T').pad().drop(columns=['customer_no']).reset_index()
    df_full['location_before'] = df_full.groupby(by=['customer_no'])['location'].shift(fill_value='entrance')

    # When the shop closes, the remaining customers are rushed through the checkout. 
    # Their checkout is not recorded, so it may look as if they stay in the market forever.
    # Here we add last transition for such customers
    last_locations = df_full.groupby(by='customer_no')[['timestamp', 'location']].last()
    missing_checkouts = last_locations[last_locations['location'] != 'checkout'].copy()
    missing_checkouts['timestamp'] = missing_checkouts['timestamp'] + pd.Timedelta(minutes=1)
    missing_checkouts['location_before'] = missing_checkouts['location']
    missing_checkouts['location'] = 'checkout'
    missing_checkouts.reset_index(inplace=True)

    return df_full.append(missing_checkouts)

df_full = resample_transitions(df)
# df_full

## Data analysis

### When customers enter the supermarket?

In [ ]:
def show_enter_time_distribution(df):
    """ plots daily avg distribution customers entering the store per hour """
    
    tmp = df.reset_index()
    days_cnt = len(df.index.day.unique())
    tmp['counter'] = 1 / days_cnt
    tmp.groupby([tmp['timestamp'].dt.hour])[['counter']].sum().plot.line()
    
show_enter_time_distribution(df)

### How long users spend in the supermarket?

In [ ]:
def show_time_in_supermarket_distribution(df):
    g = df.reset_index().groupby(['customer_no'])[['timestamp']]
    time_in_market = g.last() - g.first()
    time_in_market['counter'] = 1
    time_in_market.groupby('timestamp').count().plot()

show_time_in_supermarket_distribution(df_full)

### Probabilities plot

what are the chances to be in a specific section?

### How many sections customer visited before leaving the supermarket?

### Revenue Estimate

## Marov chain

### Transitions matrix

### Monte carlo simulations

#### Simpulation 1

#### Simpulation 2

#### Simpulation 3